In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
import random
import numpy as np 
from grid2op.Episode import EpisodeData

In [ ]:
excluded_numbers = {'29500_', '31500_'} # "31500_"

# Create a list of strings with increments of 1000 or 500, excluding the ones in the set
number_list = []

start, end = 44000, 60000

for number in range(start, end + 1):
    increment = 1000 #if number <= 29000 else 500
    if number % increment == 0:
        number_str = str(number) + "_"
        if number_str not in excluded_numbers:
            number_list.append(number_str)

print(number_list)


NB_EPISODE = 32      # number of episodes to be analyzed
lambdaa = 0.03
line_switch_cost = 0.5
RHO = 0.95    
inference_process = "inf2_"
base_directory = r"C:\Users\dwivea2\Desktop\Redis+LODF Backup Files\Try12_0.5_0.03" # Define the base directory
fileame_episode = "detailed_inference_per_episode" + "_" + str(line_switch_cost) + "_" + str(lambdaa) + ".txt"
fileame_model = "detailed_inference_per_model" + "_" + str(line_switch_cost) + "_" + str(lambdaa) + ".txt"
print("Episode Info will be stored: ", fileame_episode)
print("Model Info will be stored: ", fileame_model)

In [ ]:
overall_data = {
    "values_below_rho": [],
    "values_above_rho": [],
    "ST": [],
    "cum_RL_reward": [],
    "cum_margin_reward": [],
    "margin_to_st_ratio": [], 
    "redis_cost": []
}


for string_number in number_list:

    model = string_number
    PATH_SAVE = inference_process + model + "36bus_saved_experiment_anmol" + "_" + str(line_switch_cost) + "_" + str(lambdaa) #f"{inference_process}{model}36bus_saved_experiment_anmol_{line_switch_cost}_{lambdaa}"
    file_path = os.path.join(base_directory, PATH_SAVE)    # Construct the full file path
    print(file_path)                                       # This will display the complete file path

    all_episodes = EpisodeData.list_episode(file_path)     # a list of all episodes tested/evaluated
    print( "There are a total of ", len(all_episodes), " episodes in the evaluated folder\n" )
    
    def inspect_agent(all_episodes, which_episode):

        this_episode = EpisodeData.from_disk( *all_episodes[which_episode] ) # selecting the specific episode 
        li_actions = this_episode.actions                                    # action in theis episode object: grid2op.Episode.EpisodeData.CollectionWrapper

        with open(fileame_episode, "a") as result_file:
            result_file.write(f"Model: {model}, Episode: {which_episode}\n")

            #print( len(this_episode.rewards), np.nansum(this_episode.rewards) ) # is a numpy array, nansum because the episode might not be completed
            output_str = f"Total Time Steps: {len(this_episode.rewards)}, Sum of rewards: {np.nansum(this_episode.rewards)}\n"
            result_file.write(output_str)  # Write the formatted string to the file

        #print( this_episode.rewards )
        #print("observations: ", type(this_episode.observations) )
        #print("this_episode: ", dir(this_episode) )   #  listing all the attributes of the object
        #print("li_actions: ", type(li_actions) )

        # loading the actions chosen by the Agent, and inspecting the actions that has been taken by the agent in the current episode
        line_disc_below_rho, line_disc_above_rho = 0, 0
        redis_action_below_rho, redis_action_above_rho = 0, 0
        do_nothing_below_rho, do_nothing_above_rho = 0, 0
        line_reco_below_rho, line_reco_above_rho = 0, 0
        actions_count_below_rho, actions_count_above_rho = {}, {}

        for index, act in enumerate( li_actions ):
            obs = this_episode.observations[index]  # observation with "index" time step
            max_rho =  np.max(obs.rho)              # max-rho
            dict_ = act.as_dict()                   # type of action
            #if dict_:
            #    print("dict_: ", dict_) # this elaborates the nature of the action taken (can filter out the desired action when counting the statistics)

            if not dict_:   # do-nothing action
                if max_rho < RHO:
                    do_nothing_below_rho += 1
                else:
                    do_nothing_above_rho += 1

            elif "set_line_status" in dict_: # line switch action
                if max_rho < RHO:
                    line_reco_below_rho +=  dict_["set_line_status"]["nb_connected"]
                    line_disc_below_rho +=  dict_["set_line_status"]["nb_disconnected"]
                else:
                    line_reco_above_rho +=  dict_["set_line_status"]["nb_connected"]
                    line_disc_above_rho +=  dict_["set_line_status"]["nb_disconnected"]

            elif "redispatch" in dict_: # re-dispatch switch action
                if max_rho < RHO:
                    redis_action_below_rho += 1
                else:
                    redis_action_above_rho += 1

            if max_rho < RHO: # diversity in actions int he MDP space only: How many unique
                act_as_vect = tuple( act.to_vect() )
                if not act_as_vect in actions_count_below_rho:
                    actions_count_below_rho[act_as_vect] = 0
                actions_count_below_rho[act_as_vect] += 1
            else:
                act_as_vect = tuple( act.to_vect() )
                if not act_as_vect in actions_count_above_rho:
                    actions_count_above_rho[act_as_vect] = 0
                actions_count_above_rho[act_as_vect] += 1


        #print( len(this_episode.other_rewards), type(this_episode.other_rewards), this_episode.other_rewards[0] ) # this_episode.other_rewards is a list
        cumulative_margin_reward = [reward_dict.get("margin_reward", 0) for reward_dict in this_episode.other_rewards]
        cumulative_margin_reward = np.nansum( np.array(cumulative_margin_reward) )
        
        cumulative_redis_cost = sum(reward_dict.get("action_cost", 0) for reward_dict in this_episode.other_rewards)
        #print("cumulative_redis_cost: ", cumulative_redis_cost)
        
        with open(fileame_episode, "a") as result_file:
            #print("cumulative_margin_reward: ", cumulative_margin_reward)
            output_str = f"cumulative_margin_reward: {cumulative_margin_reward}\n"
            result_file.write(output_str)  # Write the formatted string to the file

            output_str = f"cumulative_redis_cost: {cumulative_redis_cost}\n"
            result_file.write(output_str)  # Write the formatted string to the file
        
        with open(fileame_episode, "a") as result_file:

            result_file.write(f"{which_episode}, {this_episode.meta['nb_timestep_played']}, {this_episode.meta['cumulative_reward']}\n")

            margin_to_st_ratio = cumulative_margin_reward / this_episode.meta["nb_timestep_played"]
            result_file.write(f"Margin to Survival Ratio: {margin_to_st_ratio}\n")

            def compute_percentages(a, b, c, d):
                try:
                    total = a + b + c + d
                    percentage = lambda input_number: input_number / total * 100
                    a, b, c, d = percentage(a), percentage(b), percentage(c), percentage(d)
                    return a, b, c, d
                except ZeroDivisionError:
                    # Handle the ZeroDivisionError by setting percentages to zero or some default value
                    return 0, 0, 0, 0

            result_file.write(f"Below Rho: {do_nothing_below_rho}, {line_reco_below_rho}, {line_disc_below_rho}, {redis_action_below_rho}, {len(actions_count_below_rho)}\n")
            per_do_nothing_below_rho, per_reco_below_rho, per_disc_below_rho, per_redis_below_rho = compute_percentages(do_nothing_below_rho, line_reco_below_rho, line_disc_below_rho, redis_action_below_rho)
            variables = ["Per Do nothing", "Per Reconnection", "Per Removal", "Per Redispatch"]
            values_below_rho = [per_do_nothing_below_rho, per_reco_below_rho, per_disc_below_rho, per_redis_below_rho]
            result_file.write(", ".join([f"{var}: {val}" for var, val in zip(variables, values_below_rho)]) + "\n")

            result_file.write(f"Above Rho: {do_nothing_above_rho}, {line_reco_above_rho}, {line_disc_above_rho}, {redis_action_above_rho}, {len(actions_count_above_rho)}\n")
            per_do_nothing_above_rho, per_reco_above_rho, per_disc_above_rho, per_redis_above_rho = compute_percentages(do_nothing_above_rho, line_reco_above_rho, line_disc_above_rho, redis_action_above_rho)
            variables = ["Per Do nothing", "Per Reconnection", "Per Removal", "Per Redispatch"]
            values_above_rho = [per_do_nothing_above_rho, per_reco_above_rho, per_disc_above_rho, per_redis_above_rho]
            result_file.write(", ".join([f"{var}: {val}" for var, val in zip(variables, values_above_rho)]) + "\n\n")

        return values_below_rho, values_above_rho, this_episode.meta["nb_timestep_played"], this_episode.meta["cumulative_reward"], cumulative_margin_reward, margin_to_st_ratio, cumulative_redis_cost


    # Statistics for a chosen episode.
    #chosen_episode = random.randint(0, NB_EPISODE-1)     # analyzing a random episode 

    all_values_below_rho = []
    all_values_above_rho = []
    all_ST = []
    all_cum_RL_reward = []
    all_cum_margin_reward = []
    all_margin_to_st_ratio = []
    
    all_cumulative_redis_cost = []

    for which_episode in range(NB_EPISODE):
        #which_episode = 4
        print("Episode =", which_episode)
        chosen_episode = which_episode

        values_below_rho, values_above_rho, ST, cum_RL_reward, cum_margin_reward, margin_to_st_ratio, cumulative_redis_cost = inspect_agent(all_episodes, chosen_episode)

        all_values_below_rho.append(values_below_rho)
        all_values_above_rho.append(values_above_rho)
        all_ST.append(ST)
        all_cum_RL_reward.append(cum_RL_reward)
        all_cum_margin_reward.append(cum_margin_reward)
        all_margin_to_st_ratio.append(margin_to_st_ratio)
        
        all_cumulative_redis_cost.append(cumulative_redis_cost)

    # Calculate the average of the collected values
    num_elements = len(all_values_below_rho[0])  # Assuming all lists have the same length
    average_values_below_rho = [sum(x) / len(x) for x in zip(*all_values_below_rho)]
    average_values_above_rho = [sum(x) / len(x) for x in zip(*all_values_above_rho)]
    average_ST = sum(all_ST) / len(all_ST)
    average_cum_RL_reward = sum(all_cum_RL_reward) / len(all_cum_RL_reward)
    average_cum_margin_reward = sum(all_cum_margin_reward) / len(all_cum_margin_reward)
    average_margin_to_st_ratio = sum(all_margin_to_st_ratio) / len(all_margin_to_st_ratio)
    
    average_cumulative_redis_cost = sum(all_cumulative_redis_cost) / len(all_cumulative_redis_cost)

    overall_data["values_below_rho"].append(average_values_below_rho)
    overall_data["values_above_rho"].append(average_values_above_rho)
    overall_data["ST"].append(average_ST)
    overall_data["cum_RL_reward"].append(average_cum_RL_reward)
    overall_data["cum_margin_reward"].append(average_cum_margin_reward)
    overall_data["margin_to_st_ratio"].append(average_margin_to_st_ratio)
    overall_data["redis_cost"].append(average_cumulative_redis_cost)
    
    with open(fileame_model, "a") as result_file:
        result_file.write("\n")
        result_file.write(f"model: {model}\n")
        result_file.write(f"Average Values Below Rho: {average_values_below_rho}\n")
        result_file.write(f"Average Values Above Rho: {average_values_above_rho}\n")
        result_file.write(f"Average ST: {average_ST}\n")
        result_file.write(f"Average Cumulative RL Reward: {average_cum_RL_reward}\n")
        result_file.write(f"Average Cumulative Margin Reward: {average_cum_margin_reward}\n")
        result_file.write(f"Average Margin to ST Ratio: {average_margin_to_st_ratio}\n")
        result_file.write(f"Average Redis Cost: {average_cumulative_redis_cost}\n")

        
        
average_values_below_rho = [sum(x) / len(x) for x in zip(*overall_data["values_below_rho"])]
average_values_above_rho = [sum(x) / len(x) for x in zip(*overall_data["values_above_rho"])]
average_ST = sum(overall_data["ST"]) / len(overall_data["ST"])
average_cum_RL_reward      = sum(overall_data["cum_RL_reward"]) / len(overall_data["cum_RL_reward"])
average_cum_margin_reward  = sum(overall_data["cum_margin_reward"])  / len(overall_data["cum_margin_reward"])
average_margin_to_st_ratio = sum(overall_data["margin_to_st_ratio"]) / len(overall_data["margin_to_st_ratio"])
average_cumulative_redis_cost = sum(overall_data["redis_cost"]) / len(overall_data["redis_cost"])

with open(fileame_model, "a") as result_file:
    result_file.write("\n")
    result_file.write(f"Overall Performance Average:\n")
    result_file.write(f"Average Values Below Rho: {average_values_below_rho}\n")
    result_file.write(f"Average Values Above Rho: {average_values_above_rho}\n")
    result_file.write(f"Average ST: {average_ST}\n")
    result_file.write(f"Average Cumulative RL Reward: {average_cum_RL_reward}\n")
    result_file.write(f"Average Cumulative Margin Reward: {average_cum_margin_reward}\n")
    result_file.write(f"Average Margin to ST Ratio: {average_margin_to_st_ratio}\n")
    result_file.write(f"Average Redis Cost: {average_cumulative_redis_cost}\n")